In [1]:
import numpy as np
import math
import cvxpy as cp
import pandas as pd

In [2]:
#initialization
A = pd.read_csv("A.csv")
pi = pd.read_csv("pi_2.csv")
A = np.array(A)
pi  = np.array(pi)
pi = pi.T
print(pi.shape)
n = 10000
m = 10
b_i = 1000
k = 50
b = np.array([1000 for _ in range(m)])  

x = cp.Variable(k)
s = cp.Variable(m)


(1, 10000)


Q4

In [3]:
pi1 = pi[:,:k]
A1 = A[:m,:k]
print(A[:,k])
print(A1.shape)
print(pi1.shape)
w = 1
def u(s):
    u_s = 0
    for i in range(m):
        u_s += w/m * cp.log(s[i])
    return u_s
    
obj = cp.Maximize(pi1@x + u(s))
const = [A1@x + s <= k/n * b_i, x>=0, x<=1]
prob = cp.Problem(obj, const)

prob.solve()
print(x.value)
print(prob.constraints[0].dual_value)
print("\nThe optimal value is", prob.value)
prob.solve(solver="SCS")
x.value

print("\nThe optimal value is", prob.value)
print("A solution x is")
print(x.value)
print("A dual solution corresponding to the inequality constraints is")
print(prob.constraints[0].dual_value)
print(prob.constraints[2].dual_value)



[1 0 1 0 0 1 1 0 1 1]
(10, 50)
(1, 50)
[ 4.35363331e-09  4.63716762e-01 -1.00627729e-09  9.99999972e-01
  2.22277802e-01  4.06562551e-09 -1.87989858e-09  9.99999997e-01
  3.09700836e-09  9.99999996e-01  2.09847594e-09  3.72352352e-10
 -2.25363543e-09 -1.97809831e-09 -9.57262325e-10  3.55981133e-09
  2.91181978e-08  1.82409860e-08  2.28140898e-09  5.35455489e-08
 -2.14230973e-09  3.36596837e-01 -2.32582931e-09 -2.16324721e-09
  3.16358648e-01 -2.84740204e-09 -1.67708298e-09  9.99999953e-01
  9.99999995e-01 -8.05632498e-11  2.03181910e-01 -8.84190934e-10
 -2.10239556e-09 -2.26566727e-09  4.65110390e-01  7.32680752e-08
  1.27019933e-08 -3.95328186e-10 -9.38514725e-10  9.32033087e-01
 -2.28377137e-09  9.99999995e-01  5.07794697e-01  9.99999997e-01
 -2.51651675e-09  9.99999998e-01 -2.99480879e-09 -2.45680171e-09
 -2.69603895e-09  9.99999995e-01]
[0.91382769 2.20383168 0.31988552 1.58534633 1.52407329 0.69510766
 0.1189345  2.46113792 0.05951866 2.28115224]

The optimal value is 72.013542953

Q8

In [4]:
l_decision = []
b_k = b.copy()
d = b/n
rev = 0.0
price = np.array([0.0 for _ in range(m)])
for k in range(0,n):
    # Decide whether to accept (k)th request and the remaining resource
    if pi[0][k] > A[:,k].dot(price) and np.min(b_k - A[:,k]) >= 0:
        x = 1
        b_k -=  A[:,k]
    else:
        x = 0
    l_decision.append(x)
    rev += x*pi[0][k]
    # update price first
    grad = d - A[:,k]*np.heaviside((pi[0][k]-np.dot(A[:,k],price)),0.)
    price = np.maximum(price - grad/np.sqrt(k+1), 0.)

In [5]:
x = cp.Variable(n)
ee = np.ones(n)
objective = cp.Maximize(pi@x)
constraints = [A@x <= b, x >= 0, x <= ee]
prob = cp.Problem(objective, constraints)

prob.solve()
x.value
opv1 = prob.value
print(opv1)

17406.295176501088


In [8]:
opv2 = rev
print("optimal value:", opv2)
print("competetive ratio", opv2/opv1)


optimal value: 17178.426757878282
competetive ratio 0.9869088501423076
[17178.42675788]
